In [1]:
from itertools import chain
from glob import iglob
import sqlite3

from h5py import File
import numpy as np
import pandas as pd

In [3]:
def filenames(*runs):
    for r in sorted(set(runs)):
        yield from sorted(iglob("/data/Run_{0:03d}/rawdata/Run_{0:03d}_*.h5".format(r)))

In [ ]:
runs = pd.read_excel("../Data/runs.xlsx")
bg_period, bg_mod = 3, 0

for run in runs[runs["Source"] == "N2"]["Run"]:
    break
    print("Reading Run #{}".format(run))
    for fn in filenames(run):
        try:
            with File(fn, "r") as fp:
                df = pd.DataFrame({
                    "Run": run,
                    "Tag": fp["/bunches"],
                    "Is BG": fp["/bunches"][...] % bg_period == bg_mod,
                    "IOM (pC)": fp["/photon_diagnostics/FEL01/I0_monitor/iom_sh_a_pc"],
                    "PADRES peak pos (px)": fp["/photon_diagnostics/Spectrometer/hor_spectrum"][...].argmax(1),
                    "SLU delay (ps)": (fp["/user_laser/delay_line/position"][...] * 1000).astype("int"),
                    "SLU energy (uJ)": fp["/user_laser/energy_meter/Energy1"],
                    "TOF integrated": fp["/digitizer/channel1"][...].sum(1),
                    "VMI integrated": fp["/vmi/andor"][...].sum((1, 2)),
                })
        except:
            print("Failed to read file: {}.".format(fn))
            continue

        with sqlite3.connect("file:../Data/tags.sqlite", uri=True) as con:
            df.to_sql("Tags", con, index=False, if_exists="append")

In [29]:
# Delete rows
# run = 221
# with sqlite3.connect("file:../Data/tags.sqlite", uri=True) as con:
#     c = con.cursor()
#     c.execute("DELETE FROM Tags WHERE Run = {}".format(run))

In [35]:
with sqlite3.connect("file:../Data/tags.sqlite?mode=ro", uri=True) as con:
    df = pd.read_sql("SELECT * FROM Tags WHERE Run = 270", con)
df.head()

,Run,Tag,Is BG,IOM (pC),PADRES peak pos (px),SLU delay (ps),SLU energy (uJ),TOF integrated,VMI integrated
0,270,489644200,0,9.025955,580,-6524,21.547,26518997,164752658
1,270,489644201,0,8.843994,581,-6524,22.301,26525266,156862700
2,270,489644202,1,10.194969,578,-6524,21.634,26523062,169985976
3,270,489644203,0,9.029388,582,-6524,22.881,26523524,163621004
4,270,489644204,0,9.261131,582,-6524,23.171,26525867,166803630


In [36]:
with sqlite3.connect("file:../Data/tags.sqlite?mode=ro", uri=True) as con:
    df = pd.read_sql("SELECT Run, MIN(Tag) AS 'Tag first', MAX(Tag) AS 'Tag last' "
                     "FROM Tags "
                     "GROUP BY Run", con).set_index("Run")
df.head()

,Tag first,Tag last
Run,,
212,486595947,486766228
213,486768429,486809411
214,486817329,486863766
215,486866669,486912951
216,486925290,486971619
